In [1]:
!pip3 install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!pip3 install git+https://github.com/LucknowAI/Lucknow-LLM

  Cloning https://github.com/LucknowAI/Lucknow-LLM to /tmp/pip-req-build-f4ily844
  Running command git clone --filter=blob:none --quiet https://github.com/LucknowAI/Lucknow-LLM /tmp/pip-req-build-f4ily844
  Resolved https://github.com/LucknowAI/Lucknow-LLM to commit afd5bc964a43f368ee5740644a936a1a935c875d
  Preparing metadata (setup.py) ... done
  Created wheel for lucknowllm: filename=lucknowllm-0.1-py3-none-any.whl size=506711 sha256=d0fd4abe94ab560fe922ea98692e8e1a7ae20445c31a388f9622b801f576820f
  Stored in directory: /tmp/pip-ephem-wheel-cache-20cegdof/wheels/48/3b/b7/0c5aef825590f51a81b058d149a02404a6bdaaf6fcf577c152
Successfully built lucknowllm


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from lucknowllm import UnstructuredDataLoader, split_into_segments, GeminiModel
from google.colab import userdata

In [4]:
MODEL_NAME = 'paraphrase-MiniLM-L6-v2'
API_KEY = userdata.get('gemini')
GEMINI_MODEL_NAME = "gemini-1.0-pro"
FOLDER_NAME = 'Cultural_Festival_of_Lucknow'
FILE_NAME = 'Lucknow_Mahotsav.txt'
TOP_N = 3

In [5]:
sentence_model = SentenceTransformer(MODEL_NAME)

gemini_model = GeminiModel(api_key=API_KEY, model_name=GEMINI_MODEL_NAME)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def load_and_preprocess_data():
    loader = UnstructuredDataLoader()
    external_database = loader.get_data(folder_name='Cultural_Festival_of_Lucknow', file_name='Lucknow_Mahotsav.txt')
    chunks = []
    for document in external_database:
        chunks.extend(split_into_segments(document['data']))
    return chunks


In [7]:
def embed_text_data(model, text_data):
    return model.encode(text_data)

In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b.T) / (np.linalg.norm(a, axis=1)[:, np.newaxis] * np.linalg.norm(b, axis=1))


In [9]:
def find_top_n_similar(query_vec, data_vecs, top_n=3):
    similarities = cosine_similarity(query_vec[np.newaxis, :], data_vecs)
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    return top_indices


In [10]:
def generate_gemini_response(prompt):
    return gemini_model.generate_content(prompt)


In [11]:
def main(queries):
    chunks = load_and_preprocess_data()
    embedded_data = embed_text_data(sentence_model, chunks)
    embedded_queries = embed_text_data(sentence_model, queries)

    for i, query_vec in enumerate(embedded_queries):
        top_indices = find_top_n_similar(query_vec, embedded_data, TOP_N)
        top_documents = [chunks[index] for index in top_indices]

        prompt = f"You are an expert question answering system, I'll give you a question and context, and you'll return the answer. Query: {queries[i]} Contexts: {top_documents[0]}"
        model_output = generate_gemini_response(prompt)
        print(model_output)
        return model_output


In [12]:
# Example usage
queries = ["What is the duration of Lucknow Mahotsav, and when does it usually take place?"]
res = main(queries)
print(res)

The duration of Lucknow Mahotsav is 10 days, and it usually takes place in the month of November or December.
The duration of Lucknow Mahotsav is 10 days, and it usually takes place in the month of November or December.


In [13]:
!pip3 install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

In [14]:
import gradio as gr

def lucknow_rag(input):
    return main([input])

app = gr.Interface(fn=lucknow_rag,
                   inputs="textbox",
                   outputs="textbox",
                   description="Lucknow RAG is a retrieval-augmented generation (RAG) model, which is a type of natural language processing (NLP) system that combines the strengths of retrieval-based and generation-based approaches. It utilizes a retrieval component to gather relevant information from a large corpus of text, and then employs a language generation model to produce a coherent and contextually appropriate output by conditioning on the retrieved information. The retrieval-augmented generation approach aims to leverage the broad knowledge available in large text corpora while retaining the ability to generate fluent and semantically meaningful text outputs.")
app.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9876878f9684d290c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The duration of Lucknow Mahotsav is 10 days, and it usually takes place in the month of November or December.
The duration of Lucknow Mahotsav is 10 days, and it usually takes place in the month of November or December.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9876878f9684d290c0.gradio.live
